In [ ]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from pytorch_lightning import LightningDataModule


In [ ]:
%pip install lightning
import lightning as l

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 38.0 MB/s eta 0:00:00


In [ ]:
class LightningLSTM(l.LightningModule):
    def __init__(self):
      super().__init__()
      self.lstm = nn.LSTM(input_size = 20,num_layers = 3, hidden_size = 1)

    def forward(self, df):
      st_mem, varr = self.lstm(df)
      prediction = st_mem[-1]
      return prediction

    def configure_optimizers(self):
      optimizer = optim.Adam(self.parameters(), lr=0.01)
      return optimizer

    def calculate_loss(self, input, output):
      loss_function = nn.BCEWithLogitsLoss()
      return loss_function(input, output)

    def training_step(self, batch, batch_idx):
      input_df, label_df = batch
      output_lstm = self.forward(input_df)
      loss = self.calculate_loss(output_lstm, label_df)
      return loss





In [ ]:
first_lstm_model = LightningLSTM()

In [ ]:
first_lstm_model.parameters

<bound method Module.parameters of LightningLSTM(
  (lstm): LSTM(20, 1, num_layers=3)
)>

In [ ]:
df = pd.read_csv("drive/MyDrive/neurotech/eeg_data.csv")

In [ ]:
trainer = l.Trainer(max_epochs = 10, log_every_n_steps = 5, accelerator = "gpu")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
inputs = df.iloc[:, :-1].values
labels = df.iloc[:, -1].values

inputs_tensor = torch.from_numpy(inputs).float()
labels_tensor = torch.from_numpy(labels).float()
inputs_tensor.to("cuda")
labels_tensor.to("cuda")

dataset = TensorDataset(inputs_tensor, labels_tensor)

In [ ]:
dataloader = DataLoader(dataset, shuffle = False, batch_size=375)


In [ ]:
batch_iterator = iter(dataloader)
batch = next(batch_iterator)
input, output = batch


# Process the data and target tensors here
print(input)
print(output)


tensor([[ 1.2518e+04, -7.2849e+02, -1.8749e+04, -4.4856e+03, -7.3973e+02,
         -8.3338e+03, -1.4044e+04, -2.2720e+04, -2.5476e+04, -5.9475e+04,
         -1.5328e+04, -1.3861e+04, -1.9199e+04, -1.8368e+04, -1.4253e+04,
         -2.7908e+04, -1.0400e-01,  9.6800e-01,  2.6000e-01,  1.7077e+09]])
tensor([0.])


In [ ]:
trainer.fit(first_lstm_model, train_dataloaders = dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 124   
------------------------------
124       Trainable params
0         Non-trainable params
124       Total params
0.000     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 124   
------------------------------
124       Trainable params
0         Non-trainable params
124       Total params
0.000     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([375])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([246])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
trainer.predict(first_lstm_model, inputs_tensor.unsqueeze(dim = 1))

In [ ]:
type(dataloader)

torch.utils.data.dataloader.DataLoader

In [ ]:
input = df.iloc[:, 0:19].values
input_tensor = torch.from_numpy(input)
labels = df.iloc[:, 19:20].values
labels_tensor = torch.from_numpy(labels)